# Tutorial for a GWB search on a PTA dataset with the Fourier likelihood

paper reference: *add link* [Fig. 2]

In this notebook, we present a tutorial about how to use the likelihood in Fourier domain (presented in our paper) for a GWB search on a (radio) PTA dataset. In particular, we will use the public data from EPTA DR2new (https://zenodo.org/records/8164425).

One of the main advantages of this formulation is that it allows to divide the GWB search in a two-step analysis:
- step 1: each pulsar is analyzed individually, and we focus on the signals that are not covariant with a GWB (white noise, deterministic signals, etc.). The hyperparameters of the signals covariant with a GWB are fixed to some values (in this example: log_A = -12, gamma = 5, log_kappa = -5 for both RN and DM variations).
- step 2: we look at the whole array simultaneously and carry inference for the GWB signals and for the signals covariant with it (red noise, DM variations, etc.). The signals investigated in the previous step are marginalized over.

In this example, we use the 25 pulsars of EPTA DR2new. The noise model includes white noise (efac and equad specific for each observing backend), RN and DM variations (dm_gp) for each pulsar, modeled as a gaussian process over, respectively, 30 and 100 frequency bins, and with a spectrum fitted by a flat-tail powerlaw. Furthermore, pulsar J1600-3053 presents a chromatic red noise and J1713+0747 one exponential DM dip (see the function *pulsar_model* form more details).


NB: for some enterprise extension functions this script uses a modified versions contained in the python file *sere_enterprise.py*. 

In [1]:
from __future__ import division
import sys, os, json, pickle
import numpy as np
import scipy.linalg as sl
import la_forge.gp as lfgp
import _pickle as cPickle

import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
import enterprise.constants as const
from enterprise_extensions import sampler

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import sere_enterprise as sere

Optional acor package is not installed. Acor is optionally used to calculate the effective chain length for output in the chain file.


In [2]:
####################################   FUNCTIONS    #################################### 

# build PTA object
#--------------------------------------------------------------------------------

def pulsar_model(psr, model_base, inc_chrom=True, inc_dmdip=True, fix_dip=True, fix_chrom=True):

    # chromatic noise
    if psr.name == 'J1600-3053' and inc_chrom == True:
        
        if fix_chrom == True:
            chrom = sere.chromatic_noise_block(components=93,gp_kernel='diag', psd='flat_powerlaw',gamma_val=5, logA_val=-12,logk_val=-5)
        else:
            chrom = sere.chromatic_noise_block(components=93,gp_kernel='diag', psd='flat_powerlaw')  
        model = model_base + chrom

    # DM dip
    elif psr.name == 'J1713+0747' and inc_dmdip == True:
        
        dm_expdip_tmin=[54650, 57490]
        dm_expdip_tmax=[54850, 57530]
        tmin = (dm_expdip_tmin if isinstance(dm_expdip_tmin, list)
                else [dm_expdip_tmin])
        tmax = (dm_expdip_tmax if isinstance(dm_expdip_tmax, list)
                else [dm_expdip_tmax])
        num_dmdips = 2
        dm_expdip_idx=[4,1]
        dm_expdip_idx = (dm_expdip_idx if isinstance(dm_expdip_idx,list)
                                        else [dm_expdip_idx]*int(num_dmdips))
        dm_expdip_sign='negative'
        dm_expdip_sign = (dm_expdip_sign if isinstance(dm_expdip_sign,list)
                                            else [dm_expdip_sign]*int(num_dmdips))

        if fix_dip == True:
            for dd in range(num_dmdips):
                dmdip = sere.dm_exponential_dip(tmin=tmin[dd], tmax=tmax[dd],
                                                        idx=dm_expdip_idx[dd],
                                                        sign=dm_expdip_sign[dd],
                                                        name='dmexp_{0}'.format(dd+1),
                                                        t0_dmexp_val=57510, log10_Amp_dmexp_val=-6, log10_tau_dmexp_val=1.5)
        else:
            for dd in range(num_dmdips):
                dmdip = sere.dm_exponential_dip(tmin=tmin[dd], tmax=tmax[dd],
                                                        idx=dm_expdip_idx[dd],
                                                        sign=dm_expdip_sign[dd],
                                                        name='dmexp_{0}'.format(dd+1))
        model = model_base + dmdip

    else:
        model = model_base

    return model(psr)


def build_pta (psrs, fix_rn=True, fix_dm=True, fix_wn=True, logA_red=-15, gamma_red=4, logA_dm=-16, gamma_dm=3, log_kappa=-6, inc_chrom=True, inc_dmdip=True, fix_dip=True, fix_chrom=True, inc_crn=True, hd=True):

    # find the maximum time span to set GW frequency sampling
    tmin = [p.toas.min() for p in psrs]
    tmax = [p.toas.max() for p in psrs]
    Tspan = np.max(tmax) - np.min(tmin)

    # timing model
    tm = gp_signals.MarginalizingTimingModel(use_svd=True)

    # white noise
    if fix_wn == True:
        efac = parameter.Constant()
        equad = parameter.Constant()
    else:
        efac = parameter.Uniform(0.1,5)
        equad = parameter.Uniform(-10,-5)
    ef = white_signals.MeasurementNoise(efac=efac, selection=Selection(selections.by_backend))
    eq = white_signals.TNEquadNoise(log10_tnequad=equad, selection=Selection(selections.by_backend))

    # red noise
    if fix_rn == True:
        rn_log10_A = parameter.Constant(logA_red)
        rn_gamma = parameter.Constant(gamma_red)
        rn_log10_kappa = parameter.Constant(log_kappa)
    else:
        rn_log10_A = parameter.Uniform(-18, -12)
        rn_log10_kappa = parameter.Uniform(-9,-4)
        rn_gamma = parameter.Uniform(0,7)
    rn_pl = sere.powerlaw_flat_tail(log10_A=rn_log10_A, gamma=rn_gamma, log10_kappa=rn_log10_kappa)
    rn = gp_signals.FourierBasisGP(spectrum=rn_pl, components=30, Tspan=Tspan)

    # DM variations
    if fix_dm == True:
        dm_log10_A = parameter.Constant(logA_dm)
        dm_gamma = parameter.Constant(gamma_dm) 
        dm_log10_kappa = parameter.Constant(log_kappa)
    else:
        dm_log10_A = parameter.Uniform(-18, -12)
        dm_gamma = parameter.Uniform(0,7)
        dm_log10_kappa = parameter.Uniform(-9,-4)
    pl_dm = sere.powerlaw_flat_tail(log10_A=dm_log10_A, gamma=dm_gamma, log10_kappa=dm_log10_kappa)
    dm_basis = utils.createfourierdesignmatrix_dm(nmodes=100)
    dm = gp_signals.BasisGP(priorFunction=pl_dm, basisFunction=dm_basis, name='dm')

    if hd == True:
        # HD correlated background
        crn = sere.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, components=9,  logmin=-15.5, logmax=-13.5, gamma_val=None, name='gw',orf='hd')
    else:
        # common red noise signal (uncorrelated)
        crn = sere.common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan, components=9,  logmin=-15.5, logmax=-13.5, gamma_val=None, name='crn')

    # base model
    model_base = tm + ef + eq + rn + dm

    if inc_crn == True:
        model_base += crn

    return signal_base.PTA([pulsar_model(psr,model_base,inc_chrom=inc_chrom, inc_dmdip=inc_dmdip, fix_dip=fix_dip, fix_chrom=fix_chrom) for psr in psrs])


# update covariance formula
#--------------------------------------------------------------------------------

def update_cov(Cs,ms,ns,npsrs):

    C_list = []
    n_mat = len(ns)

    for p in range(npsrs):
        C = 0
        m = 0
        for i in range(n_mat):
            C += (ns[i])*Cs[i][p][:][:] + ns[i]*ms[i][p][:].T*ms[i][p][:] 
            m += ns[i]*ms[i][p][:]

        m = m/np.sum(ns)
        C = C - np.sum(ns)*m.T*m
        C = C/(np.sum(ns))
        C_list.append(C)
        
    return C_list


In [24]:
##### FOURIER LIKELIHOOD
#--------------------------------------------------------------------------------

def log_likelihood_Fourier(xs):

    '''
        New Fourier likelihood for pta array
    '''
    params = xs if isinstance(xs, dict) else pta_fl.map_params(xs)

    phiinvs = pta_fl.get_phiinv(params, logdet=True) 

    loglike = 0
    loglike += -0.5 * np.sum([ell for ell in pta_fl.get_rNr_logdet(params)])
    loglike += sum(pta_fl.get_logsignalprior(params))

    if pta_fl._commonsignals:
        
        phiinv, logdet_phiinv = phiinvs
        
        Sigma_inv = Sigma_0_inv - phiinv_00 + phiinv
        Li, lower = sl.cho_factor(Sigma_inv, lower=True)
        
        Sigma = sl.cho_solve((Li,True), np.identity(len(Li)))
        expval = sl.cho_solve((Li,True), Si0_a_hat)
        logdet_sigma = np.sum(2 * np.log(np.diag(Li)))
        
        loglike += - 0.5 * logdet_phiinv
        loglike += 0.5 * (np.dot(Si0_a_hat, expval) - logdet_sigma)

    else:

        for Sigma_0_inv_p, pl_0, pl, Si0_a_hat_p in zip(Sigma_0_inv,phiinvs_0, phiinvs, Si0_a_hat):

            phiinv, logdet_phiinv = pl
            phiinv_0, logdet_phiinv_0 = pl_0
            
            Sigma_inv = Sigma_0_inv_p - np.diag(phiinv_0) + np.diag(phiinv)
            Li, lower = sl.cho_factor(Sigma_inv, lower=True)

            Sigma = sl.cho_solve((Li,True), np.identity(len(Li)))
            expval = sl.cho_solve((Li,True), Si0_a_hat_p)
            logdet_sigma = np.sum(2 * np.log(np.diag(Li)))

            loglike += - 0.5 * logdet_phiinv
            loglike += 0.5 * (np.dot(Si0_a_hat_p, expval) - logdet_sigma)
    
    return loglike

In [ ]:
##########################  MAIN CODE ###########################################

In [5]:
home_dir = '...'
mcmc_dir = home_dir + '...'
os.system('mkdir '+mcmc_dir)

i_logA = -12
i_gamma = 5
i_logk = -5

# reading the psrs pickle
pkl_dir = home_dir + 'psrsDR2new.pkl'
with open(pkl_dir, 'rb') as f:
    psrs = pickle.load(f)
    f.close()

Npsrs = len(psrs)
print(len(psrs))

# reading the WN dictionary
wn_dict_dir = home_dir + 'DR2new_Wnoise.json'
with open(wn_dict_dir, 'r') as f:
    Wnoise_ml = json.load(f)
    f.close()

25


### Step 1:

The first step has been carried out outside this notebook. Separately for each pulsar, we sampled over the white noise parameters (for J1713+0747 we sampled over the DM dip parameters as well), while the RN and DM variations hyperparameters were fixed (log_A = -12, gamma = 5, log_kappa = -5 for both RN and DM variations).

The obtained chains of white noise parameter samples are used to evaluate the quantities $\hat{a}_0$ and $\Sigma_0$ that are needed to construct the Fourier likelihood.

NB: when carrying out this first step, even if each pulsar is analyzed individually, the time span of the Fourier decomposition (Tspan parameter) must be defined as the Tspan of the whole array of pulsars.

### Step 2:

We sample over GWB, RN and DM hyperparameters with the Fourier likelihood:

$$\begin{aligned}
    {\rm ln} \Bigl( p(\rho, \theta |\{\delta t\}) \Bigr) & = \int {\rm d}^{N_p}a \, {\rm ln} \Bigl[p(\{ a\},\rho_0, \theta | \{\delta t\}) \times 
    \frac{p( \{a\}|\rho)p(\rho)}{p( \{a\}|\rho_0)p(\rho_0)} \Bigr] \\
    &  = \frac{1}{2} \hat{a}_0^T \Sigma_0^{-1} \Sigma \Sigma_0^{-1}\hat{a}_0 - \frac{1}{2} 
    \delta t^T \tilde{N}^{-1} \delta t - \frac{1}{2} \bigl[
    {\rm lndet}(2\pi N) + {\rm lndet}(2\pi M^TN^{-1}M) + {\rm lndet}(2\pi\phi) -
     {\rm lndet}(2\pi \Sigma) \bigr] \, ,
    \end{aligned}$$

which is Eq. (19) of our paper.

In [20]:
# building the PTA objects
pta_0 = build_pta(psrs, fix_wn=False, fix_dip=False, inc_crn=False, inc_dmdip=True, inc_chrom=True)
pta_fl = build_pta(psrs, fix_rn=False, fix_dm=False, fix_chrom=False, inc_crn=True, fix_dip=True, inc_dmdip=True, inc_chrom=True)
pta_fl.set_default_params(Wnoise_ml)

INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P200.1380_efac to 1.1045950948818555
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P217.1380_efac to 0.848194739410362
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.PS110.2487_efac to 0.9443048546615437
INFO: enterprise.signals.signal_base: Setting J0030+0451_JBO.ROACH.1520_efac to 0.9184905449812792
INFO: enterprise.signals.signal_base: Setting J0030+0451_LEAP.1396_efac to 1.4194567362018933
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.NUPPI.1484_efac to 0.776131390132143
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.NUPPI.1854_efac to 1.1089143495690343
INFO: enterprise.signals.signal_base: Setting J0030+0451_WSRT.P2.1380_efac to 0.3381260667250748
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P200.1380_log10_tnequad to -6.168262261771374
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P217.1380_log10_tnequad to -7.898145381752085
INFO:

In [9]:
# reading the Wn chains (obtained from step 1) to compute \Sigma_0 and \hat{a}_0
total_WNchain = []
check = 0
for p in psrs:
    chain_dir = home_dir + '.../chain_1.txt'
    psr_chain = np.loadtxt(chain_dir)
    inv_burn = int(0.25 * psr_chain.shape[0]) 
    if check == 0:
        total_WNchain = psr_chain[-inv_burn:,:-4]
        check = 1
    else:
        total_WNchain = np.concatenate((total_WNchain,psr_chain[-inv_burn:,:-4]),axis=1)

print('WN chains read')

WN chains read


In [25]:
# phiinv_0
phiinvs_0 = pta_0.get_phiinv([],logdet=True)

In [29]:
# Sigma_0_inv and a_hat
N = 10000
ns = np.ones(N)
lfrec = lfgp.Signal_Reconstruction(psrs, pta_0, total_WNchain, burn=0)
lfrec.reconstruct_coeffs = sere.reconstruct_coeffs
Sigma_0_inv_list = []
a_hat_list = []

for i in range(N):

    Sigma_0_inv_list_step = []
    a_hat_list_step = []
    idx = np.random.choice(np.arange(len(total_WNchain)), size=1, replace=False)
    params = lfrec.sample_posterior(idx)
    TNTs = pta_0.get_TNT(params)
    TNrs = pta_0.get_TNr(params)
    
    for TNr, TNT, pl in zip(TNrs, TNTs, phiinvs_0):
        if TNr is None:
            continue

        phiinv, logdet_phi = pl
        Sigma_0_inv_step = TNT + (np.diag(phiinv) if phiinv.ndim == 1 else phiinv)
        Li, lower = sl.cho_factor(Sigma_0_inv_step, lower=True)
        a_hat_step = sl.cho_solve((Li, True), TNr)
        Sigma_0_inv_list_step.append(Sigma_0_inv_step)
        a_hat_list_step.append(a_hat_step)

    Sigma_0_inv_list.append(Sigma_0_inv_list_step)
    a_hat_list.append(a_hat_list_step)


Sigma_0_inv = update_cov(Sigma_0_inv_list,a_hat_list,ns,Npsrs)
a_hat = [] 
for p in range(Npsrs):
    temp = 0
    for i in range(N):
        temp += a_hat_list[i][p][:]
    a_hat.append(temp/N)

In [30]:
# Sigma_0 * a hat
Si0_a_hat = []
for p in range(Npsrs):
    Si0_a_hat.append(np.dot(Sigma_0_inv[p][:][:], a_hat[p][:]))

In [31]:
if pta_fl._commonsignals:

    # phiinv_0
    phiinv_0 = []
    for i in range(Npsrs):
        phiinv_0 = np.concatenate((phiinv_0, phiinvs_0[i][0]))
    phiinv_0 = np.diag(phiinv_0)
    phiinv_00 = phiinv_0

    # Sigma_0_inv
    Sigma_0_inv = sl.block_diag(*Sigma_0_inv)

    # Sigma_0 * a hat
    Si0_a_hat_all = Si0_a_hat 
    Si0_a_hat = []
    for i in range(Npsrs):
        Si0_a_hat  = np.concatenate((Si0_a_hat , Si0_a_hat_all[i]))   


In [ ]:
# pickle up a backup for the 0 quantitities
pkl_dir = mcmc_dir + '/Sigma_0_inv.pkl'
with open(pkl_dir, "wb") as output_file:
    cPickle.dump(Sigma_0_inv, output_file)

pkl_dir = mcmc_dir + '/a_hat.pkl'
with open(pkl_dir, "wb") as output_file2:
    cPickle.dump(a_hat, output_file2)

# sampling params
np.savetxt(mcmc_dir+'/params.txt', pta_fl.param_names, fmt='%s')

In [32]:
# sampling
xs = {par.name: par.sample() for par in pta_fl.params}
print('test ln_likelihood: ',log_likelihood_Fourier(xs),'\n')

ndim = len(xs)
cov = np.diag(np.ones(ndim) * 0.01**2)
groups = None
Num = 2e6

sampler = ptmcmc(ndim, log_likelihood_Fourier, pta_fl.get_lnprior, cov, groups=groups, outDir=mcmc_dir, resume=False, seed=1234)

test ln_likelihood:  528762.4881111982 



In [ ]:
# sampling!
sampler.sample(x0, Num, SCAMweight=30, AMweight=15, DEweight=50)

With the method described in this notebook, we obtained posteriors for the hyperparameters of RN and DM variations hyperparameters for each pulsar (+ chromatic noise for J1600-3053), and for the GWB hyperparameters. The white noise and deterministic signals parameters are marginalised over. Those posteriors are compared in our paper with the ones obtained with the 'standard' time-domain likelihood in the case of a sampling over RN + DM + chrom + DM_dip + gwb (white noise parameters were fixed to reduce the computational costs). As shown in Fig.2 of our paper, the resulting posteriors for the GWB parameters are perfectly consistent.

The purpose of this notebook is to show in a simplified way how to run a GWB search with the Fourier likelihood over a (radio) PTA dataset. To produce Fig. 2 of *add link* we used this algorithm to search for a common uncorrelated red noise, and then reweighted the results according to the procedure described in Hourihane et. al 2023 (https://arxiv.org/abs/2212.06276). We also added jump proposals from the RN and DM hyperparameter prior distributions to help the sampler converge. For more details, see the python script *GWB_FourierLikelihood.py* in this same repository.

# Fourier vs time-domain: comparing apples-to-apples

We prove now with a practical example that the time- and Fourier-domain formulations of the Fourier likelihood are analytically equivalent (within numerical accuracy) in the case of white noise and deterministic signals fixed.

In [33]:
# building the PTA objects (this time, both pta_0 and pta_fl have white noise and DM dip parameters fixed)
pta_0 = build_pta(psrs, inc_crn=False, fix_dip=True, logA_red=i_logA, gamma_red=i_gamma, logA_dm=i_logA, gamma_dm=i_gamma, log_kappa=i_logk)
pta_0.set_default_params(Wnoise_ml)
pta_fl = build_pta(psrs, fix_rn=False, fix_dm=False, fix_chrom=False, inc_crn=True, fix_dip=True, inc_dmdip=True, inc_chrom=True)
pta_fl.set_default_params(Wnoise_ml)

INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P200.1380_efac to 1.1045950948818555
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P217.1380_efac to 0.848194739410362
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.PS110.2487_efac to 0.9443048546615437
INFO: enterprise.signals.signal_base: Setting J0030+0451_JBO.ROACH.1520_efac to 0.9184905449812792
INFO: enterprise.signals.signal_base: Setting J0030+0451_LEAP.1396_efac to 1.4194567362018933
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.NUPPI.1484_efac to 0.776131390132143
INFO: enterprise.signals.signal_base: Setting J0030+0451_NRT.NUPPI.1854_efac to 1.1089143495690343
INFO: enterprise.signals.signal_base: Setting J0030+0451_WSRT.P2.1380_efac to 0.3381260667250748
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P200.1380_log10_tnequad to -6.168262261771374
INFO: enterprise.signals.signal_base: Setting J0030+0451_EFF.P217.1380_log10_tnequad to -7.898145381752085
INFO:

In this case, we don't need samples of the white noise and deterministic signal parameters because tehy are fixed in the pta objects. $\sigma_0$ and $\hat{a}_0$ can be computed analytically.

In [34]:
#### ANALYTICAL \Sigma_0 and \hat{a}_0

TNTs = pta_0.get_TNT([])
TNrs = pta_0.get_TNr([])
phiinvs_0 = pta_0.get_phiinv([],logdet=True)

Sigma_0_inv = []
a_hat = []

for TNr, TNT, pl in zip(TNrs, TNTs, phiinvs_0):
    phiinv, logdet_phi = pl
    Sigma_0_inv_step = TNT + (np.diag(phiinv) if phiinv.ndim == 1 else phiinv)
    Li, lower = sl.cho_factor(Sigma_0_inv_step, lower=True)
    a_hat_step = sl.cho_solve((Li, True), TNr)
    Sigma_0_inv.append(Sigma_0_inv_step)
    a_hat.append(a_hat_step)

logdet_sigma_0 = 0
for S0 in Sigma_0_inv:
    Li, lower = sl.cho_factor(S0, lower=True)
    logdet_sigma_0 += np.sum(2 * np.log(np.diag(Li)))

# sigma 0 * a hat
Si0_a_hat = []
for p in range(Npsrs):
    Si0_a_hat.append(np.dot(Sigma_0_inv[p][:][:], a_hat[p][:]))

In [35]:
if pta_fl._commonsignals:

    # phiinv_0
    phiinv_0 = []
    for i in range(Npsrs):
        phiinv_0 = np.concatenate((phiinv_0, phiinvs_0[i][0]))
    phiinv_0 = np.diag(phiinv_0)
    phiinv_00 = phiinv_0

    # Sigma_0_inv
    Sigma_0_inv = sl.block_diag(*Sigma_0_inv)

    # Sigma_0 * a hat
    Si0_a_hat_all = Si0_a_hat 
    Si0_a_hat = []
    for i in range(Npsrs):
        Si0_a_hat  = np.concatenate((Si0_a_hat , Si0_a_hat_all[i]))  

In [36]:
# COMPARING THE LIKELIHOOD VALUE OBTAINED WITH THE TIME- AND THE FOURIER-DOMAIN LIKELIHOOD

xs = {par.name: par.sample() for par in pta_fl.params}
lnL_f = log_likelihood_Fourier(xs)
lnL_t = pta_fl.get_lnlikelihood(xs)
print('Fourier-domain ln_likelihood: ',lnL_f,'\n')
print('Time-domain ln_likelihood: ', lnL_t,'\n')
print('difference: ',lnL_f-lnL_t)

Fourier-domain ln_likelihood:  521232.07137636153 

Time-domain ln_likelihood:  521232.071376364 

difference:  -2.444721758365631e-09


In [ ]:
:)